# Prototype of lipreading pipeline

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import random
import math
import time

# model file, encoder, decoder and seqtoseq
from model_temp import *
# utils file
from utils import *
# Get landmark using vocadataset.py
from data.vocaset import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

In [ ]:
# Get landmark from vocadaset class
#trainset = vocadataset("train", landmark=True)
trainset = vocadataset("train", landmark=True, mouthOnly=True)
#landmark, labels = trainset[0]

trainloader = DataLoader(trainset, batch_size=20, collate_fn=collate_fn, shuffle=True)

#landmarks, len_landmark, label, len_label = next(iter(trainloader))


In [ ]:
vocabulary = create_vocabulary(blank="@")

In [ ]:
# Create a mapping from characters to indices
char_to_index = {char: index for index, char in enumerate(vocabulary)}

In [ ]:
# Convert the sequence and target to indices
#sequence_indices = [char_to_index[char] for char in sequence]

label_t = char_to_index_batch(label, vocabulary)

#target_indices = [char_to_index[char] for char in labels]
#target_tensor = torch.tensor(target_indices)

In [ ]:
INPUT_DIM = 36*3
INPUT_DIM = 36*3
HID_DIM = 64
output_dim = len(vocabulary)

model = only_Decoder2(INPUT_DIM, HID_DIM, 2, len(vocabulary)).to(device)



In [ ]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10000
for epoch in range(num_epochs):
    
    for i in range(landmarks.shape[0]):
        landmarks[i] = landmarks[i].to(device) 
        label_t[i] = label_t[i].to(device)
        optimizer.zero_grad()
        target_tensor = label_t[i]

        

        reshaped_landmark = torch.reshape(landmarks[i], (landmarks[i].shape[0], landmarks[i].shape[1]*landmarks[i].shape[2]))
        reshaped_landmark = reshaped_landmark.to(device)
        target_tensor = target_tensor.to(device)
        len_landmark[i] = len_landmark[i].to(device) 
        output = model(reshaped_landmark[None, : , :], len_landmark[i][None])
        output = output.permute(1,0,2)
        input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
        target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.int32)
        
        loss = ctc_loss(torch.nn.functional.log_softmax(output, dim=2), target_tensor, input_lengths, target_lengths[0])
        loss.backward()
        optimizer.step()

        e = torch.argmax(output, dim=2).squeeze(1)
        output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)
        if(epoch + 1) % 100 == 0:
            f = open("prova_.txt", "a")
            f.write(label[i]+"\n")
            f.write(output_sequence+"\n")
            f.close() 
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        #e = torch.argmax(output, dim=2).squeeze(1)
        #output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)



In [ ]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 500
for landmarks, len_landmark, label, len_label in trainloader:
    #landmarks, len_landmark, label, len_label = next(iter(trainloader))
    label_t = char_to_index_batch(label, vocabulary)
    
    for epoch in range(num_epochs):
        
        for i in range(landmarks.shape[0]):
            landmarks[i] = landmarks[i].to(device) 
            label_t[i] = label_t[i].to(device)
            optimizer.zero_grad()


            #remove padding from dataloader
            nw = landmarks[i][:len_landmark[i], :, :].clone().detach().requires_grad_(True)
            nw_l = label_t[i][:len_label[i]].clone().detach()
            
            target_tensor = nw_l
            reshaped_landmark = torch.reshape(nw, (nw.shape[0], nw.shape[1]*nw.shape[2]))
            reshaped_landmark = reshaped_landmark.to(device)
            target_tensor = target_tensor.to(device)
            len_landmark[i] = len_landmark[i].to(device) 
            output = model(reshaped_landmark[None, : , :], len_landmark[i][None])
            output = output.permute(1,0,2)
            input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
            target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.int32)
            
            loss = ctc_loss(torch.nn.functional.log_softmax(output, dim=2), target_tensor, input_lengths, target_lengths[0])
            loss.backward()
            optimizer.step()

            e = torch.argmax(output, dim=2).squeeze(1)
            output_sequence = ''.join([vocabulary[index] for index in e])
            #print(output_sequence)
            """if(epoch + 1) % 100 == 0:
                f = open("prova_.txt", "a")
                f.write(str(nw_l.tolist())+"\n")
                f.write(output_sequence+"\n")
                f.close() 
        """
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
            #e = torch.argmax(output, dim=2).squeeze(1)
            #output_sequence = ''.join([vocabulary[index] for index in e])
            #print(output_sequence)



In [ ]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


###########

#trainset = vocadataset("train", landmark=True)
trainset_ = vocadataset("train", landmark=True, mouthOnly=True)
#landmark, labels = trainset[0]
trainset_ = trainset_.to(device)
trainloader_ = DataLoader(trainset_, batch_size=1, collate_fn=collate_fn, num_workers=8)


###########à

In [ ]:
INPUT_DIM = 36*3
INPUT_DIM = 36*3
HID_DIM = 64
output_dim = len(vocabulary)

model = only_Decoder2(INPUT_DIM, HID_DIM, 2, len(vocabulary)).to(device)

In [ ]:
trainset_[0][0].shape

In [ ]:
torch.tensor(trainset_[0][0].shape[0]).shape

In [ ]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


###########
trainset_ = vocadataset("train", landmark=True, mouthOnly=True)


# Training loop
num_epochs = 10000
for epoch in range(num_epochs):

    for i  in range(10):
        optimizer.zero_grad()

        landmarks = trainset_[i][0].to(device) 
        
        label_t = char_to_index_batch(trainset_[i][1], vocabulary)
        label_t = label_t.to(device)
        target_tensor = label_t
        reshaped_landmark = torch.reshape(landmarks, (landmarks.shape[0], landmarks.shape[1]*landmarks.shape[2]))
        reshaped_landmark = reshaped_landmark.to(device)
        target_tensor = target_tensor.to(device)
        len_landmark = torch.tensor(landmarks.shape[0]).to(device) 
        output = model(reshaped_landmark[None, : , :], len_landmark[None])
        output = output.permute(1,0,2)
        input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
        target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.int32)
        
        loss = ctc_loss(torch.nn.functional.log_softmax(output, dim=2), target_tensor.squeeze(), input_lengths, target_lengths[0][None])
        loss.backward()
        optimizer.step()

        e = torch.argmax(output, dim=2).squeeze(1)
        output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)
        if(epoch + 1) % 100 == 0:
            f = open("prova_.txt", "a")
            f.write(label[i]+"\n")
            f.write(output_sequence+"\n")
            f.close() 
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        #e = torch.argmax(output, dim=2).squeeze(1)
        #output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)



In [ ]:

valset = vocadataset("val", landmark=True, mouthOnly=True)
#landmark, labels = trainset[0]

valloader = DataLoader(valset, batch_size=1, collate_fn=collate_fn)


In [ ]:


model.load_state_dict(torch.load("/home/prasanna/Documents/UNIFI/Computer Graphics/LipReading/lipreading/models/model_f__5500_7.pt"))
model.eval()

real_sentences = []
pred_sentences = []

with torch.no_grad():

    for landmarks, len_landmark, label, len_label in valloader:

        # reshape the batch from [batch_size, frame_size, num_landmark, 3] to [batch_size, frame_size, num_landmark * 3] 
        landmarks = torch.reshape(landmarks, (landmarks.shape[0], landmarks.shape[1], landmarks.shape[2]*landmarks.shape[3]))
        
        #variable to recover later the target sequences
        label_list = label

        # label char to index
        label = char_to_index_batch(label, vocabulary)

        # move data to GPU!
        landmarks = landmarks.to(device)
        len_landmark = len_landmark.to(device)
        label = label.to(device)
        len_label = len_label.to(device)

        output = model(landmarks, len_landmark)
        output = output.permute(1, 0, 2)
        # scrittura nel file del outuput e della frase originale

        real_sentences, pred_sentences = write_results(len_label, label_list, output.detach(), valloader.batch_size, vocabulary, real_sentences, pred_sentences)

    pred_sentences = list(map(lambda x:process_string(x),pred_sentences))
    save_results(f"./results/validation.txt", real_sentences, pred_sentences, overwrite=True)



In [ ]:
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction


def calculate_exact_match(model_output, ground_truth):
    return int(model_output == ground_truth)


def calculate_word_error_rate(model_output, ground_truth):
    model_words = model_output.split()
    ground_truth_words = ground_truth.split()
    len_model = len(model_words)
    len_gt = len(ground_truth_words)

    dp = [[0] * (len_gt + 1) for _ in range(len_model + 1)]

    for i in range(len_model + 1):
        dp[i][0] = i

    for j in range(len_gt + 1):
        dp[0][j] = j

    for i in range(1, len_model + 1):
        for j in range(1, len_gt + 1):
            if model_words[i - 1] == ground_truth_words[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

    return dp[len_model][len_gt] / len_gt


def calculate_bleu(model_output, ground_truth):
    smoothie = SmoothingFunction().method1
    return sentence_bleu([ground_truth], model_output, smoothing_function=smoothie)


def calculate_rouge(model_output, ground_truth, metric="rouge-l"):
    rouge = Rouge()
    scores = rouge.get_scores(model_output, ground_truth)
    return scores[0][metric]["f"]

def calcualteMetrics(predictList, ground_truth_list):
    em_score = 0
    wer_score = 0
    bleu_score = 0
    rouge_score = 0
    for i in range(len(predictList)):
        model_output = predictList[i]
        ground_truth = ground_truth_list[i]
        em_score += calculate_exact_match(model_output, ground_truth)
        wer_score += calculate_word_error_rate(model_output, ground_truth)
        bleu_score += calculate_bleu(model_output.split(), ground_truth.split())
        rouge_score += calculate_rouge(model_output, ground_truth, metric="rouge-l")
    
    em_score = em_score/len(predictList)*100
    wer_score= wer_score/len(predictList)*100
    bleu_score = bleu_score/len(predictList)*100
    rouge_score = rouge_score/len(predictList)*100
    
    return em_score, wer_score, bleu_score, rouge_score


In [ ]:
def plot_metrics(em_score, wer_score, bleu_score, rouge_score):

    metrics = ["EM", "WER", "BLEU", "ROUGE"]
    scores = [em_score, wer_score, bleu_score, rouge_score]

    plt.bar(metrics, scores)
    plt.ylim(0, 1)
    plt.ylabel("Score")
    plt.title("Evaluation Metrics")
    plt.show()


# Example usage
model_output = "The cat is on the mat"
ground_truth = "There is a cat on the mat"

#plot_metrics(model_output, ground_truth)


In [ ]:
em_score, wer_score, bleu_score, rouge_score = calcualteMetrics(pred_sentences, real_sentences)
plot_metrics(em_score, wer_score, bleu_score, rouge_score)
em_score, wer_score, bleu_score, rouge_score